In [ ]:
# Busca a entropia em eV e J/mol.k nos arquivos de saida

def get_entropy_of(filename):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()

    entropy = {}
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]
    
    for line_number,line in enumerate(lines_bottom):
        if 'Entropy' in line:
            entropy ['S_vib (eV)'] = line.split()[2]
            entropy ['S_vib (J/mol.K)'] = lines_bottom[line_number+1].split()[1]
    return(entropy)

print(get_entropy_of("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out"))

# Busca a energia livre de Gibbs em eV e kJ/mol.k nos arquivos de saida

def get_final_free_energy(filename):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()

    final_free_energy = {}
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]

    for line_number,line in enumerate(lines_bottom):
      if 'Total free energy :' in line:
        final_free_energy ['Final free energy (eV)'] = lines_bottom[line_number+1].split()[4]
      if 'Total free energy (in kJmol-1)' in line:
        final_free_energy ['Final free energy (kJ/mol.K)'] = lines_bottom[line_number+1].split()[4]
        
    return(final_free_energy)

print(get_final_free_energy("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out"))

# Busca a constante dielétrica estática nos arquivos de saida

def get_sdlc_of(filename):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()

    sdlc = {}
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]

    for line_number,line in enumerate(lines_bottom):
        if 'Static dielectric constant tensor :' in line:
          for i in (1 ,2 ,3 ):
            for j in (1, 2, 3):
              sdlc [f'sdlc{i}{j}'] = lines_bottom[line_number + 4 + i].split()[j]

    return(sdlc)

print(get_sdlc_of("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out"))

{'S_vib (eV)': '0.000678', 'S_vib (J/mol.K)': '65.435130'}
{'Final free energy (eV)': '-24.86106012', 'Final free energy (kJ/mol.K)': '-2398.7123'}
{'sdlc11': '9.10439', 'sdlc12': '-0.00000', 'sdlc13': '-0.00000', 'sdlc21': '-0.00000', 'sdlc22': '9.10439', 'sdlc23': '-0.00000', 'sdlc31': '-0.00000', 'sdlc32': '-0.00000', 'sdlc33': '9.10439'}


In [ ]:
def get_static_dielectric_constant_tensor_of(filename):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()

    static_dielectric_constant_tensor = {}

    for line_number,line in enumerate(lines):
        if 'Static dielectric constant tensor :' in line:
          static_dielectric_constant_tensor ['sdct_11'] = lines[line_number+5].split()[1]
          static_dielectric_constant_tensor ['sdct_12'] = lines[line_number+5].split()[2]
          static_dielectric_constant_tensor ['sdct_13'] = lines[line_number+5].split()[3]
          static_dielectric_constant_tensor ['sdct_21'] = lines[line_number+6].split()[1]
          static_dielectric_constant_tensor ['sdct_22'] = lines[line_number+6].split()[2]
          static_dielectric_constant_tensor ['sdct_23'] = lines[line_number+6].split()[3]
          static_dielectric_constant_tensor ['sdct_31'] = lines[line_number+7].split()[1]
          static_dielectric_constant_tensor ['sdct_32'] = lines[line_number+7].split()[2]
          static_dielectric_constant_tensor ['sdct_33'] = lines[line_number+7].split()[3]

    return(static_dielectric_constant_tensor)

print(get_static_dielectric_constant_tensor_of("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out"))

In [ ]:
def get_sdlc_of(filename):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()

    sdlc = {}

    for line_number,line in enumerate(lines):
        if 'Static dielectric constant tensor :' in line:
          for i in (1 ,2 ,3 ):
            for j in (1, 2, 3):
              sdlc [f'sdlc{i}{j}'] = lines[line_number + 4 + i].split()[j]

    return(sdlc)

print(get_sdlc_of("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out"))

{'sdlc11': '9.10439', 'sdlc12': '-0.00000', 'sdlc13': '-0.00000', 'sdlc21': '-0.00000', 'sdlc22': '9.10439', 'sdlc23': '-0.00000', 'sdlc31': '-0.00000', 'sdlc32': '-0.00000', 'sdlc33': '9.10439'}


# Ultimo arquivo alterado

In [ ]:
# Busca a entropia em eV e J/mol.k nos arquivos de saida

def get_properties(filename):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()
    
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]
    
    entropy = {}
    
    for line_number,line in enumerate(lines_bottom):
        if 'Entropy' in line:
            entropy ['S_vib (eV)'] = line.split()[2]
            entropy ['S_vib (J/mol.K)'] = lines_bottom[line_number+1].split()[1]

# Busca a energia livre de Gibbs em eV e kJ/mol.k nos arquivos de saida

    final_free_energy = {}

    for line_number,line in enumerate(lines_bottom):
      if 'Total free energy :' in line:
        final_free_energy ['Final free energy (eV)'] = lines_bottom[line_number+1].split()[4]
      if 'Total free energy (in kJmol-1)' in line:
        final_free_energy ['Final free energy (kJ/mol.K)'] = lines_bottom[line_number+1].split()[4]
        
    return (entropy, final_free_energy)

print(get_properties("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out"))


({'S_vib (eV)': '0.000678', 'S_vib (J/mol.K)': '65.435130'}, {'Final free energy (eV)': '-24.86106012', 'Final free energy (kJ/mol.K)': '-2398.7123'})


* Código de teste

In [ ]:
# Busca a entropia em eV e J/mol.k nos arquivos de saida

def get_entropy_of(filename,wanted_properties):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()
    
    properties = {}

    entropy = {}
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]
    
    for line_number,line in enumerate(lines_bottom):
        if 'Entropy' in line:
            entropy ['S_vib (eV)'] = line.split()[2]
            entropy ['S_vib (J/mol.K)'] = lines_bottom[line_number+1].split()[1]
    
  # Busca a energia livre de Gibbs em eV e kJ/mol.k nos arquivos de saida

    final_free_energy = {}

    for line_number,line in enumerate(lines_bottom):
      if 'Total free energy :' in line:
        final_free_energy ['Final free energy (eV)'] = lines_bottom[line_number+1].split()[4]
      if 'Total free energy (in kJmol-1)' in line:
        final_free_energy ['Final free energy (kJ/mol.K)'] = lines_bottom[line_number+1].split()[4]
    
    if 'entropy' in wanted_properties:
      properties.update(entropy)

    if 'final_free_energy' in wanted_properties:
      properties.update(final_free_energy)
    return(properties)

print(get_entropy_of("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out", ['entropy','final_free_energy']))

{'S_vib (eV)': '0.000678', 'S_vib (J/mol.K)': '65.435130', 'Final free energy (eV)': '-24.86106012', 'Final free energy (kJ/mol.K)': '-2398.7123'}


* Codigo de teste 20.07.2020

In [ ]:
def get_entropy_of(filename,wanted_properties):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()
    
    properties = {}
    
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]
    
    if 'bulk_modulus' in wanted_properties:

        bulk_modulus = {}

        for line_number,line in enumerate(lines_bottom):
            if 'Mechanical properties :' in line:
                bulk_modulus[f'bulk_modulus_Reuss'] = lines_bottom[line_number + 5].split()[4]
                bulk_modulus[f'bulk_modulus_Voigt'] = lines_bottom[line_number + 5].split()[5]
                bulk_modulus[f'bulk_modulus_Hill'] = lines_bottom[line_number + 5].split()[6]
        
        properties.update(bulk_modulus)

    return(properties)

print(get_entropy_of("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out",['bulk_modulus']))

{'bulk_modulus_Reuss': '46.44211', 'bulk_modulus_Voigt': '46.44211', 'bulk_modulus_Hill': '46.44211'}


In [ ]:
def get_entropy_of(filename,wanted_properties):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()
    
    properties = {}
    
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]
    
    if 'bulk_modulus' in wanted_properties:

        bulk_modulus = {}
        convention = ('Reuss', 'Voigt', 'Hill')

        for line_number,line in enumerate(lines_bottom):
            if 'Mechanical properties :' in line:
                for bk in convention:
                    bulk_modulus[f'bulk_modulus_{bk}'] = lines_bottom[line_number + 5].split()[4+convention.index(bk)]
        
        properties.update(bulk_modulus)

    return(properties)

print(get_entropy_of("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out",['bulk_modulus']))

{'bulk_modulus_Reuss': '46.44211', 'bulk_modulus_Voigt': '46.44211', 'bulk_modulus_Hill': '46.44211'}


# Ultima alteração 20.07.2020

In [ ]:
# Busca a entropia em eV e J/mol.k nos arquivos de saida

def get_properties(filename,wanted_properties):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()
    
    properties = {}
    
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]

    if 'entropy' in wanted_properties:

        entropy = {}
   
        for line_number,line in enumerate(lines_bottom):
            if 'Entropy' in line:
                entropy ['S_vib (eV)'] = line.split()[2]
                entropy ['S_vib (J/mol.K)'] = lines_bottom[line_number+1].split()[1]
        properties.update(entropy)
    
# Busca a energia livre de Gibbs em eV e kJ/mol.k nos arquivos de saida    
    
    if 'final_free_energy' in wanted_properties:
        
        final_free_energy = {}

        for line_number,line in enumerate(lines_bottom):
            if 'Total free energy :' in line:
                final_free_energy ['Final free energy (eV)'] = lines_bottom[line_number+1].split()[4]
            if 'Total free energy (in kJmol-1)' in line:
                final_free_energy ['Final free energy (kJ/mol.K)'] = lines_bottom[line_number+1].split()[4]
    
        properties.update(final_free_energy)

# Busca as constantes dielétrica estatica 
    
    if 'sdlcij' in wanted_properties:

        sdlc = {}

        for line_number,line in enumerate(lines_bottom):
            if 'Static dielectric constant tensor :' in line:
                for i in (1 ,2 ,3 ):
                    for j in (1, 2, 3):
                        sdlc [f'sdlc{i}{j}'] = lines_bottom[line_number + 4 + i].split()[j]
        
        properties.update(sdlc)

# Busca as constantes dielétrica de alta frequência 

    if 'hfdlcij' in wanted_properties:

        hfdlc = {}

        for line_number,line in enumerate(lines_bottom):
            if 'High frequency dielectric constant tensor :' in line:
                for i in (1 ,2 ,3 ):
                    for j in (1, 2, 3):
                        hfdlc[f'hfdlc{i}{j}'] = lines_bottom[line_number + 4 + i].split()[j]
        
        properties.update(hfdlc)

# Busca as constantes elásticas em Gpa 

    if 'Cij' in wanted_properties:

        Cij = {}

        for line_number,line in enumerate(lines_bottom):
            if 'Elastic Constant Matrix: (Units=GPa)' in line:
                for i in (1 ,2 ,3, 4, 5, 6):
                    for j in (1 ,2 ,3, 4, 5, 6):
                        Cij[f'C{i}{j}'] = lines_bottom[line_number + 4 + i].split()[j]
        
        properties.update(Cij)

# Busca o módulo de bulk

    if 'bulk_modulus' in wanted_properties:

        bulk_modulus = {}
        convention = ('Reuss', 'Voigt', 'Hill')

        for line_number,line in enumerate(lines_bottom):
            if 'Mechanical properties :' in line:
                for bk in convention:
                    bulk_modulus[f'bulk_modulus_{bk}'] = lines_bottom[line_number + 5].split()[4+convention.index(bk)]
        
        properties.update(bulk_modulus)

    return(properties)

print(get_properties("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out",['bulk_modulus']))

{'bulk_modulus_Reuss': '46.44211', 'bulk_modulus_Voigt': '46.44211', 'bulk_modulus_Hill': '46.44211'}


# Codigo de teste 03.08.2020

Não sei se tu notou, mas cada propriedade ou componente de matriz pode ser acessada definindo três coisas:

• Uma expressão a ser procurada no arquivo

• Quantas linhas após essa expressão está o que a gente quer

• Em qual campo da linha está o que a gente quer

A gente vai usar isso pra simplificar ainda mais a função e generalizar ela.
Tu vai definir um dicionário de dicionários com esses parâmetros:

***param = {prop1:{'expression':string1, 'after':number_of_lines1, 'field':field_number1}, prop2:{'expression':string2, 'after':number_of_lines2, 'field':field_number2}, ...}***

A chave do dicionário principal vai ser o nome da propriedade. O valor vai ser um dicionário secundário, que vai ter 3 chaves, um pra cada parâmetro.

Aí, pra cada propriedade tu só precisa ir nesse dicionário pra saber os parâmetros pra a busca dessa propriedade

Pra pegar o número do campo da propriedade, por exemplo, tu faz:

***field = param[prop]['field']***

Isso vai enxugar bastante a função

In [ ]:

def get_properties(filename,wanted_properties):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()
    
    properties = {}
    
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]

    parameters = {'S_eV':{'expression':'Entropy','after':0,'field':2},
                  'S_J/molK':{'expression':'Entropy','after':1,'field':1},
                  'G_eV':{'expression':'Total free energy :','after':1,'field':4},
                  'G_kJ/molK':{'expression':'Total free energy (in kJmol-1)','after':1,'field':4},
                  **{f'sdlc{i}{j}':{'expression':'Static dielectric constant tensor :','after':4+i,'field':j} for i in (1,2,3) for j in (1,2,3)},
                  **{f'hfdlc{i}{j}':{'expression':'High frequency dielectric constant tensor :','after':4+i,'field':j} for i in (1,2,3) for j in (1,2,3)},
                  **{f'C{i}{j}':{'expression':'Elastic Constant Matrix: (Units=GPa)','after':4+i,'field':j} for i in (1,2,3,4,5,6) for j in (1,2,3,4,5,6)},
                  **{f'bulk_modulus_{i}':{'expression':'Mechanical properties :','after':5,'field':4+('Reuss', 'Voigt', 'Hill').index(i)} for i in ('Reuss', 'Voigt', 'Hill')}}

# Busca a entropia em eV e J/mol.k nos arquivos de saida

    if 'entropy' in wanted_properties:

        entropy = {}
   
        for line_number,line in enumerate(lines_bottom):
            if 'Entropy' in line:
                entropy ['S_vib (eV)'] = line.split()[2]
                entropy ['S_vib (J/mol.K)'] = lines_bottom[line_number+1].split()[1]
        properties.update(entropy)
    
# Busca a energia livre de Gibbs em eV e kJ/mol.k nos arquivos de saida    
    
    if 'final_free_energy' in wanted_properties:
        
        final_free_energy = {}

        for line_number,line in enumerate(lines_bottom):
            if 'Total free energy :' in line:
                final_free_energy ['Final free energy (eV)'] = lines_bottom[line_number+1].split()[4]
            if 'Total free energy (in kJmol-1)' in line:
                final_free_energy ['Final free energy (kJ/mol.K)'] = lines_bottom[line_number+1].split()[4]
    
        properties.update(final_free_energy)

# Busca as constantes dielétrica estatica 
    
    if 'sdlcij' in wanted_properties:

        sdlc = {}

        for line_number,line in enumerate(lines_bottom):
            if 'Static dielectric constant tensor :' in line:
                for i in (1 ,2 ,3 ):
                    for j in (1, 2, 3):
                        sdlc [f'sdlc{i}{j}'] = lines_bottom[line_number + 4 + i].split()[j]
        
        properties.update(sdlc)

# Busca as constantes dielétrica de alta frequência 

    if 'hfdlcij' in wanted_properties:

        hfdlc = {}

        for line_number,line in enumerate(lines_bottom):
            if 'High frequency dielectric constant tensor :' in line:
                for i in (1 ,2 ,3 ):
                    for j in (1, 2, 3):
                        hfdlc[f'hfdlc{i}{j}'] = lines_bottom[line_number + 4 + i].split()[j]
        
        properties.update(hfdlc)

# Busca as constantes elásticas em Gpa 

    if 'Cij' in wanted_properties:

        Cij = {}

        for line_number,line in enumerate(lines_bottom):
            if 'Elastic Constant Matrix: (Units=GPa)' in line:
                for i in (1 ,2 ,3, 4, 5, 6):
                    for j in (1 ,2 ,3, 4, 5, 6):
                        Cij[f'C{i}{j}'] = lines_bottom[line_number + 4 + i].split()[j]
        
        properties.update(Cij)

# Busca o módulo de bulk

    if 'bulk_modulus' in wanted_properties:

        bulk_modulus = {}
        convention = ('Reuss', 'Voigt', 'Hill')

        for line_number,line in enumerate(lines_bottom):
            if 'Mechanical properties :' in line:
                for bk in convention:
                    bulk_modulus[f'bulk_modulus_{bk}'] = lines_bottom[line_number + 5].split()[4+convention.index(bk)]
        
        properties.update(bulk_modulus)

    return(properties)

print(get_properties("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out",['hfdlcij','bulk_modulus']))

{'hfdlc11': '2.83901', 'hfdlc12': '-0.00000', 'hfdlc13': '-0.00000', 'hfdlc21': '-0.00000', 'hfdlc22': '2.83901', 'hfdlc23': '-0.00000', 'hfdlc31': '-0.00000', 'hfdlc32': '-0.00000', 'hfdlc33': '2.83901', 'bulk_modulus_Reuss': '46.44211', 'bulk_modulus_Voigt': '46.44211', 'bulk_modulus_Hill': '46.44211'}


Rodolpho, [03.08.20 14:04]
Bacana

Rodolpho, [03.08.20 14:04]
Agora dá pra usar esse dicionário na função.

Rodolpho, [03.08.20 14:05]
O usuário passa uma lista das propriedades que ele quer

Rodolpho, [03.08.20 14:05]
E tu faz um loop entre os elementos dessa lista

Rodolpho, [03.08.20 14:06]
Pra cada elemento, tu procura essa propriedade no dicionário e pega a expression, o after e o field

Rodolpho, [03.08.20 14:06]
E usa isso pra achar o valor no output

Rodolpho, [03.08.20 14:07]
Aí tu adiciona o nome da propriedade e o valor ao dicionário das coisas que tu vai retornar para o usuário.

Rodolpho, [03.08.20 14:07]
E o loop passa para a próxima iteração.

Mas veja que o usuário vai te passar uma lista de propriedades. E tu vai retornar um dicionário de propriedades. Com cada propriedade e seu respectivo valor.

# Ultima alteração 23.09.2020

In [35]:
def get_properties(filename,wanted_properties):
    with open(filename,'r') as output_file:
        lines = output_file.readlines()
    
    properties = {}
    
    final_G_index = [line_number for line_number,line in enumerate(lines) if 'Final free energy' in line]
    lines_bottom = lines[final_G_index[0]:]

    parameters = {'S_eV':{'expression':'Entropy','after':0,'field':2},
                  'S_J/molK':{'expression':'Entropy','after':1,'field':1},
                  'G_eV':{'expression':'Total free energy :','after':1,'field':4},
                  'G_kJ/molK':{'expression':'Total free energy (in kJmol-1)','after':1,'field':4},
                  **{f'sdlc{i}{j}':{'expression':'Static dielectric constant tensor :','after':4+i,'field':j} for i in (1,2,3) for j in (1,2,3)},
                  **{f'hfdlc{i}{j}':{'expression':'High frequency dielectric constant tensor :','after':4+i,'field':j} for i in (1,2,3) for j in (1,2,3)},
                  **{f'C{i}{j}':{'expression':'Elastic Constant Matrix: (Units=GPa)','after':4+i,'field':j} for i in (1,2,3,4,5,6) for j in (1,2,3,4,5,6)},
                  **{f'bulk_modulus_{i}':{'expression':'Mechanical properties :','after':5,'field':4+('Reuss', 'Voigt', 'Hill').index(i)} for i in ('Reuss', 'Voigt', 'Hill')}}

    for element in wanted_properties:
        expression = parameters[element]['expression']
        after = parameters[element]['after']
        field = parameters[element]['field']  
        for line_number,line in enumerate(lines_bottom):
            if expression in line: 
                properties[wanted_properties[wanted_properties.index(element)]] = lines_bottom[line_number+after].split()[field]

    return(properties)

print(get_properties("/content/drive/My Drive/Colab Notebooks/Li2S_New(300.0).out",['S_eV','S_J/molK','G_eV','G_kJ/molK','sdlc11','hfdlc11','C11','bulk_modulus_Reuss']))

{'S_eV': '0.000678', 'S_J/molK': '65.435130', 'G_eV': '-24.86106012', 'G_kJ/molK': '-2398.7123', 'sdlc11': '9.10439', 'hfdlc11': '2.83901', 'C11': '96.5770', 'bulk_modulus_Reuss': '46.44211'}
